In [1]:
import os
import numpy as np
from scipy.misc import imread
from matplotlib import pyplot as plt
from skimage import transform
import xml.etree.ElementTree as ET
from glob import glob
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [2]:
# Load the pretrained model
model = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

In [3]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [4]:
data_path = 'VOCtrainval_06-Nov-2007'
data = glob(os.path.join(data_path,"*","*","*","*.jpg"))
file = data[0]

In [7]:
def get_vector(image):
    # 1. Load the image with Pillow library
    img = Image.open(image)
#     img = Image.fromarray(image)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
#         print(o.data[0,:,0,0].shape)
        my_embedding.copy_(o.data[0,:,0,0])
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

In [9]:
get_vector(file)

torch.Size([512])